# Init

In [1]:
import sys
import os
import shutil
import logging
# DEBUG: Detailed information, typically of interest only when diagnosing problems.
# INFO: Confirmation that things are working as expected.
# WARNING: An indication that something unexpected happened, or indicative of some problem in the near future (e.g. ‘disk space low’). The software is still working as expected.
# ERROR: Due to a more serious problem, the software has not been able to perform some function.
# CRITICAL: A serious error, indicating that the program itself may be unable to continue running.
import json
import pandas as pd
import numpy as np
import string
import random
import math
import time
import re
from datetime import datetime
from datetime import date
from datetime import timedelta
from datetime import timezone
from pytz import timezone
import pytz
import pause
import humanfriendly
from bs4 import BeautifulSoup
from tabulate import tabulate
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from flask import Flask, jsonify, request, url_for
from flask import Flask, session
from flask_session import Session
import requests
working_time_zone='Europe/Kiev'
alpari_date_format="%m/%d/%Y"
fh_time_format="%Y-%m-%d %H:%M:%S"
fh_date_format='%Y-%m-%d'
os.environ['TZ'] = working_time_zone
time.tzset()
work_dir=os.getcwd()
log_path=os.path.dirname(work_dir)
log_file=os.path.join(log_path, os.path.basename(log_path) + '.log')
open(log_file, 'w').close()
os.chmod(log_file, 0o775)
def echo(msg, function_name="", level="info"):
    if function_name=="":
        logger = logging.getLogger('pnews')
    else:
        logger = logging.getLogger('pnews' + '.' + function_name)
    if level=="info":
        logger.info(msg)
    if level=="warn":
        logger.warning(msg)
    if level=="error":
        logger.error(msg)
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger('')
logger.handlers = []
try:
    logger = logging.getLogger('pnews')
    logger.handlers = []
    fh = logging.FileHandler(log_file)
    fh.setLevel(logging.INFO)
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(message)s', 
                                  datefmt=fh_time_format)
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    logger.addHandler(fh)
    logger.addHandler(ch)
    logger.info('service started')
except Exception as e:
        print("Exception " + str(e))

2019-01-28 19:41:13:INFO:pnews:service started


In [2]:
app = Flask(__name__)
app.debug = True
SESSION_TYPE = 'filesystem'
app.config.from_object(__name__)
Session(app)

# Flask routes

In [3]:
# Main page
# curl -i http://localhost:80/pnews/
@app.route('/')
def flask_index():
    echo("main page opened.", "pnews")
    return "Service pnews (language python) is ready!\r\n"
# Health check
# curl -i http://localhost:80/pnews/health
@app.route('/health')
def flask_health():
    echo("checked.", "health")
    return 'Healthy\r\n'
# Echo
# curl -i http://localhost:80/pnews/echo?msg=hello
@app.route('/echo', methods=['GET'])
def flask_echo(): 
    msg=request.args.get('msg', '')
    echo(msg, "echo")
    return jsonify(msg='The message is: "' + msg + '"\r\n')
# Pause 
# curl -i http://localhost:80/pnews/pause/2
@app.route('/pause/<int:duration>', methods=['GET'])
def flask_pause(duration):
    echo("Pause " + str(duration) + " seconds", "pause")
    dt=datetime.now()+timedelta(seconds=duration)
    pause.until(dt)
    return "Pause of " + str(duration) + ' seconds finished\r\n'
@app.route('/open_browser')
def flask_open_browser():
    driver, text=open_browser()
    session["driver"]=driver
    return "Browser opened\r\n" + text
@app.route('/close_browser')
def flask_close_browser():
    close_browser(session["driver"])
    return "Browser closed\r\n"
@app.route('/test_connectivity')
def flask_test_connectivity():
    pass

In [4]:
with app.test_request_context():
    print(url_for('flask_echo'))
    print(url_for('flask_close_browser'))
    print(url_for('flask_close_browser', next='/'))
    print(url_for('flask_pause', duration=2))

/echo
/close_browser
/close_browser?next=%2F
/pause/2


# Testing

In [5]:
def create_test_df():
    date_rng = pd.date_range(start='2019-01-24', end='2019-01-25', freq='H')
    l=len(date_rng)
    dts = pd.to_datetime(date_rng, format=fh_time_format)  
    hashes=[''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(3)) for x in range(l)]
    ns=list(range(l))
    vs=random.sample(range(100), l)
    d={'dt': dts, "event":hashes, "number": ns, "value": vs}
    df=pd.DataFrame(d)
    df['dt']=[dt.strftime('%Y-%m-%d %H:%M:%S') for dt in df.dt]
    return df

In [6]:
def write_request_json(out_file='test.json'):
    df=create_test_df()
    df.head()
    df.to_json(out_file)
    df1=pd.read_json(out_file)
    df1.head()

In [7]:
def read_response_json(in_file='response.json'):
    df=pd.read_json(in_file)
    df.head()

# Browser automation

In [8]:
def open_browser(sleep_time=10):
    print ('start opening browser')
    cap = DesiredCapabilities.CHROME
    driver = webdriver.Remote(command_executor='http://selenium-hub:4444/wd/hub',desired_capabilities=cap) 
    url="https://www.google.com"
    print ('Opening URL: ' + url + ' waiting approximately ' + str(sleep_time))
    driver.get(url)
    wait = WebDriverWait(driver, sleep_time)
    element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.gLFyf')))    
    print ('browser opened')
    text=driver.find_element_by_xpath("html").text
    print (text)
    return driver, text
def close_browser(driver):
    print ('closing chrome driver')
    driver.quit()
    print ('driver closed')

# Main

In [ ]:
try:
    if __name__ == '__main__':
            app.run(host='0.0.0.0', port=5000) # open url http://0.0.0.0:5000/
except Exception as e:
    print('Exception starting flask app: ' + str(e))
pass 

In [ ]:
!echo $GIT_VERSION

In [9]:
#session = requests.Session()
#session.trust_env = False
url='http://pnews:5000'
resp = requests.get(url)
#ret = session.get(url)
print(resp)

<Response [200]>
